In [1]:
# Importing all libraries - you do not require any other libraries

# Common libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datasets import load_dataset
import seaborn as sns


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import seaborn as sns

In [2]:
# Load the LIAR dataset
dataset = load_dataset("liar")

cols_to_remove = ["id", "barely_true_counts", "false_counts", "half_true_counts", "mostly_true_counts", "pants_on_fire_counts"]
dataset = dataset.remove_columns(cols_to_remove)

df_train = pd.DataFrame(dataset["train"])
df_test = pd.DataFrame(dataset["test"])


# Extract features (X) and target variable (y)
#X = dataset["train"].remove_columns(["label"])  # Features
#X = pd.DataFrame(X)
#X = pd.get_dummies(X)

#y = dataset["train"]["label"]  # Target variable
#y = pd.DataFrame(y)


Found cached dataset liar (/Users/meglarkin/.cache/huggingface/datasets/liar/default/1.0.0/479463e757b7991eed50ffa7504d7788d6218631a484442e2098dabbf3b44514)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
nltk_stopwords = set(stopwords.words('english'))

def PreprocessSentence(sentence, nltk_stopwords):
    # Input - a sentence in the form of a python string, and the set of stopwords
    # Output - the preprocessed sentence, as per the instructions
    sentence = sentence.lower()
    tokens = word_tokenize(sentence)
    print(tokens)
    no_stopword = [word for word in tokens if word not in nltk_stopwords and (len(word) > 1 or word.isnumeric())]
    sentence = ' '.join(no_stopword)

    return sentence
    

def PreprocessData(df, nltk_stopwords):
    # Input - the dataframe, with columns label and text, and the set of stopwords
    # output - the dataframe with the text processed as described earlier
    df["statement"] = df["statement"].map(lambda x: PreprocessSentence(x, nltk_stopwords))
    
    return df

In [4]:
df_train = PreprocessData(df_train, nltk_stopwords)
df_test = PreprocessData(df_test, nltk_stopwords)


['says', 'the', 'annies', 'list', 'political', 'group', 'supports', 'third-trimester', 'abortions', 'on', 'demand', '.']
['when', 'did', 'the', 'decline', 'of', 'coal', 'start', '?', 'it', 'started', 'when', 'natural', 'gas', 'took', 'off', 'that', 'started', 'to', 'begin', 'in', '(', 'president', 'george', 'w.', ')', 'bushs', 'administration', '.']
['hillary', 'clinton', 'agrees', 'with', 'john', 'mccain', '``', 'by', 'voting', 'to', 'give', 'george', 'bush', 'the', 'benefit', 'of', 'the', 'doubt', 'on', 'iran', '.', "''"]
['health', 'care', 'reform', 'legislation', 'is', 'likely', 'to', 'mandate', 'free', 'sex', 'change', 'surgeries', '.']
['the', 'economic', 'turnaround', 'started', 'at', 'the', 'end', 'of', 'my', 'term', '.']
['the', 'chicago', 'bears', 'have', 'had', 'more', 'starting', 'quarterbacks', 'in', 'the', 'last', '10', 'years', 'than', 'the', 'total', 'number', 'of', 'tenured', '(', 'uw', ')', 'faculty', 'fired', 'during', 'the', 'last', 'two', 'decades', '.']
['jim', 'd

In [5]:
df_train.statement

0        says annies list political group supports thir...
1        decline coal start started natural gas took st...
2        hillary clinton agrees john mccain `` voting g...
3        health care reform legislation likely mandate ...
4                     economic turnaround started end term
                               ...                        
10264    larger number shark attacks florida cases vote...
10265     democrats become party atlanta metro area blacks
10266    says alternative social security operates galv...
10267      lifting u.s. cuban embargo allowing travel cuba
10268    department veterans affairs manual telling vet...
Name: statement, Length: 10269, dtype: object

In [27]:
#def transform_value(value):
#   if value in [1, 2, 3]:
#        return 1
#    elif value in [0, 4, 5]:
#        return 0
#    else:
#        return value

#df["label"] = df["label"].apply(transform_value)

In [6]:
df_train.label

0        0
1        1
2        2
3        0
4        1
        ..
10264    2
10265    2
10266    1
10267    0
10268    5
Name: label, Length: 10269, dtype: int64

In [7]:
def vectorize(df):
    # Input - the dataframe with columns label and text
    # output - the dataframe with text replaced by tf-idf vectors, a numpy array of labels, and the vectorizer 
   # Vectorization
    text = df["statement"]
    labels = df["label"].to_numpy()
    vectorizer = TfidfVectorizer()
    transformFit = vectorizer.fit_transform(text)
    vectorizer.get_feature_names_out()

    # Transformation and feature extraction

    newDataFrame = pd.DataFrame(transformFit.toarray(), columns=vectorizer.get_feature_names_out())
    return newDataFrame, labels, vectorizer

In [8]:
X, y, vectorizer = vectorize(df_train)

In [9]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train the Decision Tree Model
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

# Make predictions
y_pred = dt_classifier.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# You can also print more detailed metrics using classification_report
print(classification_report(y_test, y_pred))

Accuracy: 0.21616358325219084
              precision    recall  f1-score   support

           0       0.21      0.28      0.24       391
           1       0.25      0.24      0.25       448
           2       0.23      0.24      0.23       370
           3       0.20      0.20      0.20       348
           4       0.19      0.15      0.17       334
           5       0.16      0.13      0.15       163

    accuracy                           0.22      2054
   macro avg       0.21      0.21      0.21      2054
weighted avg       0.22      0.22      0.21      2054



In [10]:
def predictOnSample(sample_text, dt_classifier, vectorizer):
    # Inputs : the sample text to classify, the logistic regression model and the tdf-idf vectorizer
    # outputs: A string - "Negative" or "Positive", corresponding to the prediction
    clean = PreprocessSentence(sample_text, nltk_stopwords)
    vectorized_input = vectorizer.transform([clean])
    return dt_classifier.predict(vectorized_input)

In [11]:
sample_1 = "Says that Tennessee law requires that schools receive half of proceeds -- $31 million per year -- from a half-cent increase in the Shelby County sales tax"
pred= predictOnSample(sample_1, dt_classifier, vectorizer)
print(pred)

['says', 'that', 'tennessee', 'law', 'requires', 'that', 'schools', 'receive', 'half', 'of', 'proceeds', '--', '$', '31', 'million', 'per', 'year', '--', 'from', 'a', 'half-cent', 'increase', 'in', 'the', 'shelby', 'county', 'sales', 'tax']
[2]


/usr/local/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [12]:
# Get feature importances
feature_importances = dt_classifier.feature_importances_

# Create a DataFrame to display feature importances
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print or visualize feature importances
print(importance_df)

           Feature  Importance
9568          says    0.027898
8057       percent    0.012643
11995        years    0.008583
8052        people    0.008481
7523         obama    0.008264
...            ...         ...
4335       fathers    0.000000
4336        faucet    0.000000
4337         fault    0.000000
4338        faulty    0.000000
12050  zuckerbergs    0.000000

[12051 rows x 2 columns]
